In [ ]:
import cnn
import xarray as xr
import numpy as np
import torch
import torch.nn as nn
import einops
import matplotlib.pyplot as plt


In [ ]:
from sklearn.utils import shuffle
from tqdm import tqdm
from IPython.display import clear_output


In [ ]:
model = cnn.cnn_class()

In [ ]:
ds = xr.open_dataset('/home/ronja/Praktikum-IFU/example_data.nc', engine='netcdf4')

In [ ]:
ds

In [ ]:
last_hour = range(-1,-60,-1)
ds2 = (ds.radar)>0.1
ds['wet_radar'] = ds2[:,last_hour].sum('timestep')

# ds['wet_radar'] = ((ds.radar*60)>0.1).isel(timestep=-5)


In [ ]:
ds['wet_radar']

In [ ]:
def balance_classes(a, boo):
    lsn=len(a.sample_number)
    ind = np.arange(lsn)
    ind_true = shuffle(ind[boo])
    ind_false = ind[~boo]
    ind_true = ind_true[:np.sum(~boo)]
    print(1-(2*len(ind_false)/lsn))
    return a.isel(sample_number=np.concatenate([ind_true,ind_false]))

In [ ]:
ds = balance_classes(ds, ~ds.wet_radar.values)

In [ ]:
ds['tl'] = ds.tl.transpose('sample_number', 'channel_id', 'timestep')

In [ ]:
ds

In [ ]:
batchsize = 10
working_size = int(len(ds.sample_number)*0.8/batchsize)* batchsize

In [ ]:
working_size

In [ ]:
tl, wr = shuffle(ds.tl.values, ds.wet_radar.values)

data = tl[:working_size]
data_test = tl[working_size:]
data_out = wr[:working_size]
data_test_out = wr[working_size:]

In [ ]:
np.histogram(data_test_out)

bsp zum "abrunden" auf ganze batch size
`data = data[:int(len(data/batch_size))*batch_size]`

In [ ]:

### data as Tensor
data = torch.Tensor(data)
data_test = torch.Tensor(data_test)

data_out = torch.Tensor(data_out)
data_test_out = torch.Tensor(data_test_out)


In [ ]:

### prepare data
dataset = torch.utils.data.TensorDataset(data, data_out)
testset = torch.utils.data.TensorDataset(data_test, data_test_out)
trainloader = torch.utils.data.DataLoader(dataset, batch_size = batchsize, shuffle = True)
testloader = torch.utils.data.DataLoader(testset, batch_size = batchsize, shuffle = False)


In [ ]:
epochs = 50
resume = 0

In [ ]:
if resume == 0:
    loss_dict = {}
    loss_dict['train'] = {}
    loss_dict['test'] = {}
    for key in ['train','test']:
        loss_dict[key]['loss'] = []

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


In [ ]:
for epoch in range(resume, epochs):
    # training
    train_losses = []
    for inputs, targets in tqdm(trainloader):
        optimizer.zero_grad()
        # print("inputs_size = ", inputs.size())
        pred = model(inputs)
        pred = nn.Flatten(0,1)(pred)
        loss = nn.BCELoss()(pred, targets)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.detach().numpy())
    loss_dict['train']['loss'].append(np.mean(train_losses))
    
    # testing
    test_losses = []
    with torch.no_grad():
        # print("testloader_size: ", len(testloader))
        for inputs, targets in tqdm(testloader):
            pred = model(inputs)
            pred = pred.round()
            pred = nn.Flatten(0,1)(pred)
            loss = nn.BCELoss()(pred, targets)
            test_losses.append(loss.detach().numpy())
            # print("loss: ", loss)
        loss_dict['test']['loss'].append(np.mean(test_losses))
        
    # printing
    clear_output(wait=True)
    print(epoch)
    print('')
    print('train loss:', np.mean(train_losses))
    print('test loss:', np.mean(test_losses))
    print('min test loss:', np.min(loss_dict['test']['loss']))
    fig, axs = plt.subplots(1,1, figsize=(3,4))
    for key in loss_dict.keys():
        for k, key2 in enumerate(loss_dict[key].keys()):
            axs.plot(loss_dict[key][key2], label=key)
            axs.set_title(key2)
    # axs.set_yscale('log')
    plt.legend()
    plt.show()
    resume = epoch

In [ ]:
for inputs, targets in tqdm(trainloader):
    print(model(inputs))
    break